In [1]:
!pip install requests
!pip install numpy
!pip install pandas
!pip install selenium
!pip install webdriver_manager

In [65]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import requests
import re
import numpy as np
import pandas as pd
import time
import random


Reference: [here](https://www.geeksforgeeks.org/google-maps-selenium-automation-using-python/)

In [39]:
malls = []
driver = webdriver.Chrome()
driver.get("https://id.wikipedia.org/wiki/Daftar_pusat_perbelanjaan_di_Jakarta")
mall_s = driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/div[1]/ul/li')
for mall in mall_s:
    malls.append(mall.text)
driver.quit()
malls = [mall.replace('Mall', 'Mal').replace('Mal', 'Mall') for mall in malls]

In [41]:
krls = []
driver = webdriver.Chrome()
driver.get("https://id.wikipedia.org/wiki/Kategori:Stasiun_kereta_api_di_Jakarta")
krl_s = driver.find_elements(By.XPATH, '//*[@id="mw-pages"]/div/div/div/ul/li/a')
for krl in krl_s:
    krls.append(krl.text)
driver.quit()

In [56]:
tjs = []
driver = webdriver.Chrome()
driver.get("https://id.wikipedia.org/wiki/Daftar_koridor_Transjakarta")
tj_s = driver.find_elements(By.CSS_SELECTOR, 'tr>td:nth-child(2)>a.mw-redirect[href^="/wiki/"]')
for tj in tj_s:
    tjs.append('Halte Busway ' + tj.text)
driver.quit()
tjs = sorted(set(tjs))

In [60]:
rss = []
driver = webdriver.Chrome()
driver.get("https://id.wikipedia.org/wiki/Daftar_rumah_sakit_di_DKI_Jakarta")
rs_s = driver.find_elements(By.CSS_SELECTOR, 'tr>td:nth-child(3)>a.new[href]')
for rs in rs_s:
    rss.append(rs.text)
driver.quit()

In [63]:
list_to_scrape = malls + krls + tjs + rss
len(list_to_scrape)

519

In [73]:
url_s = ['']*len(list_to_scrape)
stars = ['']*len(list_to_scrape)
count_reviews = ['']*len(list_to_scrape)
driver = webdriver.Chrome()
for i in range(len(list_to_scrape)):
    driver.get("https://www.google.com/maps/@-6.208428,106.7824432,15z")
    time.sleep(1.81 + np.random.normal(loc=0.0,scale=0.327))
    Place = driver.find_element(By.CLASS_NAME, "tactile-searchbox-input")
    Place.send_keys(list_to_scrape[i])
    time.sleep(2.08 + np.random.normal(loc=0.0,scale=0.327))
    Place.send_keys(Keys.DOWN)
    time.sleep(0.15)
    Submit = driver.find_element(By.ID, "searchbox-searchbutton")
    Submit.click()
    time.sleep(2 + np.random.normal(loc=0.0,scale=0.127))
    try:
        url_s[i] = driver.current_url
        stars[i] = driver.find_element(By.CSS_SELECTOR, 'span[aria-label^="Bintang"]').accessible_name # bintang = star
        count_reviews[i] = driver.find_element(By.CSS_SELECTOR, 'span[aria-label$="lasan"]').accessible_name # ulasan = review
    except:
        pass

In [74]:
# check yang reviewnya kurang dari 10
# check yang nama sama urlnya beda banget

In [75]:
df = pd.DataFrame(data = {
    'name': list_to_scrape
    , 'url': url_s
    , 'stars': stars
    , 'count_reviews': count_reviews
})

In [76]:
df.head()

,name,url,stars,count_reviews
0,Central Park Jakarta,https://www.google.com/maps/place/Central+Park...,"Bintang 4,7",96.897 ulasan
1,Ciplaz Cengkareng,https://www.google.com/maps/place/Ciplaz+Cengk...,"Bintang 4,6",2.441 ulasan
2,CityWalk Gajah Mada,https://www.google.com/maps/place/Citywalk+Gaj...,"Bintang 4,5",2.978 ulasan
3,Green Sedayu Mall,https://www.google.com/maps/place/Green+Sedayu...,"Bintang 4,6",3.745 ulasan
4,HubLife Taman Anggrek,https://www.google.com/maps/place/Hublife+Tama...,"Bintang 4,6",2.225 ulasan


In [77]:
df.to_csv('gmaps_scrape_data.csv')

In [78]:
# code 2023-04-26 sampai sini

In [11]:
df['coordinate'] = df['url'].apply(lambda x: re.search(r'@([-\d\.,]+)z', x)[1])
df['stars'] = df['stars'].apply(lambda x: re.search(r'^Bintang\s([\d,]+)', str(x))[1] if re.search(r'^Bintang\s([\d,]+)', str(x)) else None)
df['count_reviews'] = df['count_reviews'].apply(lambda x: re.search(r'([\d\.]+)\s.lasan$', str(x))[1] if re.search(r'([\d\.]+)\s.lasan$', str(x)) else None)

In [12]:
df.head()

,name,url,stars,count_reviews,coordinate
0,Central Park Jakarta,https://www.google.com/maps/place/Central+Park...,"4,7",96.878,"-6.1773686,106.7880974,17"
1,Ciplaz Cengkareng,https://www.google.com/maps/place/Ciplaz+Cengk...,"4,6",292,"-6.1529989,106.7256858,17"
2,CityWalk Gajah Mada,https://www.google.com/maps/search/CityWalk+Ga...,"4,5",2.978,"-6.1453085,106.8126383,17"
3,Green Sedayu Mall,https://www.google.com/maps/place/Green+Sedayu...,"4,6",3.744,"-6.1388522,106.7261465,17"
4,HubLife Taman Anggrek,https://www.google.com/maps/search/HubLife+Tam...,"4,6",2.225,"-6.1805602,106.7898675,17"


In [18]:
df['stars'] = df['stars'].str.replace(',', '.').astype('float64')

In [21]:
df.isna().sum()

name              0
url               0
stars            97
count_reviews    97
coordinate        0
dtype: int64

In [23]:
df[ df['stars'].isna() ]

,name,url,stars,count_reviews,coordinate
5,Lindeteves Trade Centre,https://www.google.com/maps/place/Lindeteves+T...,NaN,None,"-6.1452508,106.7971913,14"
9,Mal Taman Anggrek,https://www.google.com/maps/place/Taman+Anggre...,NaN,None,"-6.1785831,106.7896379,17"
37,Cilandak Town Square,"https://www.google.com/maps/place/Town+Square,...",NaN,None,"-6.2914315,106.7966429,17"
51,Pacific Place,https://www.google.com/maps/search/Pacific+Pla...,NaN,None,"-6.208428,106.7824432,15"
55,Plaza Kalibata,https://www.google.com/maps/place/PLAZA+KALIBA...,NaN,None,"-6.2572996,106.8534192,17"
...,...,...,...,...,...
494,RS Yadika Kebayoran Lama,"https://www.google.com/maps/dir/-6.2097506,106...",NaN,None,"-6.2325016,106.7482327,13"
498,RSIA Kartini,https://www.google.com/maps/place/Kartini+Hosp...,NaN,None,"-6.208428,106.7824432,15"
532,RS Mediros,https://www.google.com/maps/place/Rumah+Sakit+...,NaN,None,"-6.1798748,106.901146,17"
533,RS Ridwan Meuraksa,https://www.google.com/maps/search/RS+Ridwan+M...,NaN,None,"-6.208428,106.7824432,15"


Somehow, for some malls, there are duplicates with different name. One of them has the rating, while some are not. For example:
1. Mal Taman Anggrek
2. Mall Taman Anggrek

Since malls and hospitals are important information and there is actually nothing we can do, I will patch all data manually.

However, for the stations, I will impute the rows with generic value.

In [ ]:
df.at[5, 'stars'] = 4.6
df.at[9, 'stars'] = 4.6
df.at[37, 'stars'] = 4.5
df.at[51, 'stars'] = 4.7
df.at[55, 'stars'] = 4.4
df.at[60, 'stars'] = 4.4
df.at[61, 'stars'] = 4.3
df.at[62, 'stars'] = 4.4
df.at[63, 'stars'] = 4.4
df.at[64, 'stars'] = 4.4
df.at[67, 'stars'] = 4.4
df.at[70, 'stars'] = 4.3
df.at[71, 'stars'] = 4.5
df.at[72, 'stars'] = 4.4
df.at[73, 'stars'] = 4.4
df.at[74, 'stars'] = 4.3
df.at[75, 'stars'] = 4.5

In [32]:
df.at[5, 'count_reviews'] = '3.447'
df.at[9, 'count_reviews'] = '59.507'
df.at[37, 'count_reviews'] = '43.861'
df.at[51, 'count_reviews'] = '45.256'
df.at[55, 'count_reviews'] = '3.120'
df.at[60, 'count_reviews'] = '11.239'
df.at[61, 'count_reviews'] = '11.061'
df.at[62, 'count_reviews'] = '32.776'
df.at[63, 'count_reviews'] = '5.931'
df.at[64, 'count_reviews'] = '15.674'
df.at[65, 'count_reviews'] = '7.291'
df.at[67, 'count_reviews'] = '3.765'
df.at[70, 'count_reviews'] = '3.298'
df.at[71, 'count_reviews'] = '14.052'
df.at[72, 'count_reviews'] = '1.268'
df.at[73, 'count_reviews'] = '22.341'
df.at[74, 'count_reviews'] = '7.623'
df.at[75, 'count_reviews'] = '31.063'

In [34]:
df.at[65, 'coordinate'] = '-6.1867509,106.9563119,17' # grand cakung vs grand cakung mall

In [33]:
df[ df['stars'].isna() ].head(20)

,name,url,stars,count_reviews,coordinate
66,Lippo Plaza Kramat Jati,https://www.google.com/maps/search/Lippo+Plaza...,NaN,None,"-6.208428,106.7824432,15"
68,Mall Cipinang Indah,https://www.google.com/maps/place/Mall+Cipinan...,NaN,None,"-6.208428,106.7824432,15"
76,Mal Artha Gading,https://www.google.com/maps/place/Mall+Artha+G...,NaN,None,"-6.208428,106.7824432,15"
77,Mal Kelapa Gading,https://www.google.com/maps/search/Mal+Kelapa+...,NaN,None,"-6.208428,106.7824432,15"
78,Mall Of Indonesia,https://www.google.com/maps/search/Mall+Of+Ind...,NaN,None,"-6.208428,106.7824432,15"
79,Metro Sunter Plaza,https://www.google.com/maps/search/Metro+Sunte...,NaN,None,"-6.208428,106.7824432,15"
80,PIK Avenue,https://www.google.com/maps/place/PIK+Avenue/@...,NaN,None,"-6.208428,106.7824432,15"
81,Pluit Junction,https://www.google.com/maps/search/Pluit+Junct...,NaN,None,"-6.1189586,106.7748871,15"
82,Pluit Village,https://www.google.com/maps/search/Pluit+Villa...,NaN,None,"-6.1160125,106.7852917,17"
83,Sunter Mall,https://www.google.com/maps/place/Sunter+Mall/...,NaN,None,"-6.137975,106.8682928,17"


In [35]:
df.at[66, 'stars'] = 4.4
df.at[68, 'stars'] = 4.4
df.at[76, 'stars'] = 4.5
df.at[77, 'stars'] = 4.6
df.at[78, 'stars'] = 4.5
df.at[79, 'stars'] = 4.5
df.at[80, 'stars'] = 4.6
df.at[81, 'stars'] = 4.3
df.at[82, 'stars'] = 4.4
df.at[83, 'stars'] = 4.5

In [36]:
df.at[66, 'count_reviews'] = '20.524'
df.at[68, 'count_reviews'] = '22.795'
df.at[76, 'count_reviews'] = '49.957'
df.at[77, 'count_reviews'] = '75.001'
df.at[78, 'count_reviews'] = '49.231'
df.at[79, 'count_reviews'] = '1.987'
df.at[80, 'count_reviews'] = '19.587'
df.at[81, 'count_reviews'] = '1.185'
df.at[82, 'count_reviews'] = '89'
df.at[83, 'count_reviews'] = '10.290'

In [37]:
df[ df['stars'].isna() ].tail(20)

,name,url,stars,count_reviews,coordinate
347,Halte Busway Terminal Tanjung Priok,https://www.google.com/maps/search/Halte+Buswa...,NaN,None,"-6.1235806,106.8618246,14"
352,Halte Busway Fly Over Raden Inten,https://www.google.com/maps/dir/Slipi+Petambur...,NaN,None,"-6.2295523,106.713879,11"
356,Halte Busway Stasiun Kalender,https://www.google.com/maps/dir/Slipi+Petambur...,NaN,None,"-6.2234155,106.7673579,12"
357,Halte Busway Cipinang,https://www.google.com/maps/search/Halte+Buswa...,NaN,None,"-6.2084024,106.6279276,11"
360,Halte Busway Fly Over Jatinegara,https://www.google.com/maps/dir/Slipi+Petambur...,NaN,None,"-6.223938,106.7565302,12"
362,Halte Busway Rumah Sakit Primier,https://www.google.com/maps/dir/Slipi+Petambur...,NaN,None,"-6.2345931,106.6798282,12"
369,Halte Busway Kalibesar Barat,https://www.google.com/maps/dir/Slipi+Petambur...,NaN,None,"-6.1688766,106.7595493,13"
370,Halte Busway Pekojan,https://www.google.com/maps/search/Halte+Buswa...,NaN,None,"-6.134461,106.8021457,17"
371,Halte Busway Kemayoran Landasan Pacu Timur,https://www.google.com/maps/dir/Slipi+Petambur...,NaN,None,"-6.208428,106.7824432,15"
376,Halte Busway Kip,https://www.google.com/maps/search/Halte+Buswa...,NaN,None,"-6.2084263,106.751543,13"


In [ ]:
df.at[433, 'stars'] = 3 # no data
df.at[456, 'stars'] = 4.4
df.at[494, 'stars'] = 4.5
df.at[77, 'stars'] = 4.6
df.at[78, 'stars'] = 4.5
df.at[79, 'stars'] = 4.5
df.at[80, 'stars'] = 4.6
df.at[81, 'stars'] = 4.3
df.at[82, 'stars'] = 4.4
df.at[83, 'stars'] = 4.5

In [ ]:
df.at[433, 'count_reviews'] = 5 # no data. Following Laplace, I will give 1 for every rating
df.at[456, 'count_reviews'] = 4.7
df.at[494, 'count_reviews'] = 4.1
df.at[77, 'count_reviews'] = 4.6
df.at[78, 'count_reviews'] = 4.5
df.at[79, 'count_reviews'] = 4.5
df.at[80, 'count_reviews'] = 4.6
df.at[81, 'count_reviews'] = 4.3
df.at[82, 'count_reviews'] = 4.4
df.at[83, 'count_reviews'] = 4.5

In [ ]:
df.at[456, 'coordinate'] = '-6.2487219,106.783051,17'
df.at[494, 'coordinate'] = '-6.2551136,106.7750808,17'